In [53]:
import pandas as pd
from datetime import date
import calendar
import pprint
import numpy as np
import sys
import os
import glob as glob

pd.options.display.max_columns = 999
pd.options.display.max_rows = 2000

#Accounting Software Control Sheet
control_sheet_df= pd.DataFrame()
control_sheet_data = glob.glob('/Users/rginsberg/Downloads/Subway-Paul Chopra/Accounting Software Control Sheet/*')

for file in control_sheet_data:
    df = pd.read_excel(file)
    df.columns = df.iloc[2]
    df.drop(df.index[0:3],inplace=True)
    df = df.reset_index(drop=True)
    control_sheet_df = control_sheet_df.append(df)

control_sheet_df = control_sheet_df.reset_index(drop=True)
control_sheet_df = control_sheet_df.infer_objects()


#Royalty/FAF/IPC Data Frame
roy_faf_ipc_df = control_sheet_df.copy(deep=True)
roy_faf_ipc_df = roy_faf_ipc_df[['Store Number',
                               'Date',
                               'Cashcards',
                               'Cash Card Sales',
                               'Catering Call Center',
                               'PayPal',
                               'Unit Sales',
                               'Drinks Sales',
                               'Misc Sales' ]]

roy_faf_ipc_df= roy_faf_ipc_df.groupby(['Store Number'], as_index=False).sum()
roy_faf_ipc_df['Total Sales'] = roy_faf_ipc_df['Unit Sales'] + roy_faf_ipc_df['Drinks Sales'] + roy_faf_ipc_df['Misc Sales']
roy_faf_ipc_df['Royalty'] = roy_faf_ipc_df['Total Sales'] * .08
roy_faf_ipc_df['Faf'] = roy_faf_ipc_df['Total Sales'] * .045
roy_faf_ipc_df['Commission'] = roy_faf_ipc_df['Cash Card Sales'] * .025
roy_faf_ipc_df['Fees'] = ((roy_faf_ipc_df['Catering Call Center'] * .07) + (roy_faf_ipc_df['Cashcards'] * .025) + (roy_faf_ipc_df['PayPal'] * .02))
roy_faf_ipc_df['Net IPC'] = roy_faf_ipc_df['Catering Call Center'] + roy_faf_ipc_df['Cashcards'] - roy_faf_ipc_df['Cash Card Sales'] + roy_faf_ipc_df['Commission'] - roy_faf_ipc_df['Fees'] + roy_faf_ipc_df['PayPal']
roy_faf_ipc_df['Cash Card Sales Commission'] = roy_faf_ipc_df['Cash Card Sales'] * .025
roy_faf_ipc_df['Cash Card Redeemed Fee 2.5%'] = roy_faf_ipc_df['Cashcards'] * .025
roy_faf_ipc_df['Paypal Fee 2%'] = roy_faf_ipc_df['PayPal'] * .02
roy_faf_ipc_df['7% Call Center'] = roy_faf_ipc_df['Catering Call Center'] * .07
roy_faf_ipc_df = roy_faf_ipc_df.round(2)


#Report Group
report_group_df = pd.DataFrame()
report_group_data = glob.glob('/Users/rginsberg/Downloads/Subway-Paul Chopra/Accounting Report Group/*')

for file in report_group_data:
    df = pd.read_excel(file)
    df.columns = df.iloc[5]
    df.drop(df.index[0:6],inplace=True)
    df = df.reset_index(drop=True)
    df = df[~df.Store.str.contains("Total")]
    df = df[~df.Store.str.contains("GRAND TOTAL")]
    report_group_df = report_group_df.append(df)
    
    
#Delivery Data DF    
delivery_data_df = report_group_df.copy(deep=True)
delivery_data_df = delivery_data_df[['Date',
                                     'Store',
                                     'Tender Amex',
                                     'Tender APP',
                                     'Tender Cash',
                                     'Tender CashCard',
                                     'Tender Catering Center',
                                     'Tender DEL-DOORDASH',
                                     'Tender DEL-Grubhub',
                                     'Tender DEL-Postmates',
                                     'Tender DEL-UberEatS',
                                     'Tender Discover',
                                     'Tender EBT',
                                     'Tender MasterCard',
                                     'Tender No Tender Type',
                                     'Tender PayPal',
                                     'Tender RewardsCard',
                                     'Tender VISA']]
delivery_data_df['Daily Delivery Sum'] = delivery_data_df['Tender DEL-DOORDASH'] + delivery_data_df['Tender DEL-Grubhub'] + delivery_data_df['Tender DEL-Postmates'] + delivery_data_df['Tender EBT']
delivery_data_df['Other'] = delivery_data_df['Tender EBT']

trnsid = control_sheet_df['Date']
delivery_trnsid = delivery_data_df['Date']
class_list = control_sheet_df['Store Number']
delivery_class_list = delivery_data_df['Store']
date_list = control_sheet_df['Date']
delivery_date_list = delivery_data_df['Date']

df_batchimport = pd.DataFrame({'TRNSID': trnsid,
                               'Amount': 0.00,
                               'Memo': 'Batch Import',
                               'Class' : class_list,
                               'TRNSTYPE': 'GENERAL JOURNAL',
                               'Date': date_list,
                               '!TRNS': 'Yes'})
df__delivery_batchimport = pd.DataFrame({'TRNSID': delivery_trnsid,
                               'Amount': 0.00,
                               'Memo': 'Batch Import',
                               'Class' : delivery_class_list,
                               'TRNSTYPE': 'GENERAL JOURNAL',
                               'Date': delivery_date_list,
                               '!TRNS': 'Yes'})

df_deposits = pd.DataFrame({'TRNSID': trnsid,
                            'Amount': control_sheet_df['Deposit In Bank'],
                            'Memo': 'Cash Deposits', 
                            'Class' : class_list, 
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': date_list})


df_gift_card_catering = pd.DataFrame({'TRNSID': trnsid,
                                      'Amount': (control_sheet_df['Cashcards'] - control_sheet_df['Cash Card Sales']) + control_sheet_df['Catering Call Center'], 
                                      'Memo': 'Gift Card plus Catering',
                                      'Class' : class_list,
                                      'TRNSTYPE': 'GENERAL JOURNAL',
                                      'Date': date_list})

df_vmd_credit_sales = pd.DataFrame({'TRNSID': trnsid,
                                    'Amount': control_sheet_df['Visa And Mastercard'] + control_sheet_df['Discover'],
                                    'Memo': 'VMD Credit Sales',
                                    'Class' : class_list,
                                    'TRNSTYPE': 'GENERAL JOURNAL',
                                    'Date': date_list})

df_amex_credit_sales = pd.DataFrame({'TRNSID': trnsid,
                                     'Amount': control_sheet_df['American Express'],
                                     'Memo': 'Amex Credit Sales',
                                     'Class' : class_list,
                                     'TRNSTYPE': 'GENERAL JOURNAL',
                                     'Date': date_list})

df_paid_outs = pd.DataFrame({'TRNSID': trnsid,
                             'Amount': control_sheet_df['Paidouts'],
                             'Memo': 'Paid Outs',
                             'Class' : class_list,
                             'TRNSTYPE': 'GENERAL JOURNAL',
                             'Date': date_list})

df_paypal_1 = pd.DataFrame({'TRNSID': trnsid,
                            'Amount': control_sheet_df['PayPal'],
                            'Memo': 'PayPal',
                            'Class' : class_list,
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': date_list})


df_over_short = pd.DataFrame({'TRNSID': trnsid,
                              'Amount': control_sheet_df['Over Short'] * -1,
                              'Memo': 'Over/Short',
                              'Class' : class_list,
                              'TRNSTYPE': 'GENERAL JOURNAL',
                              'Date': date_list})

df_opencash = pd.DataFrame({'TRNSID': trnsid,
                            'Amount': control_sheet_df['Opening Cash'],
                            'Memo': 'Open Cash',
                            'Class' : class_list,
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': date_list})

df_closekeep = pd.DataFrame({'TRNSID': trnsid,
                             'Amount': control_sheet_df['Closing Keep'] * -1,
                             'Memo': 'Close Keep',
                             'Class' : class_list,
                             'TRNSTYPE': 'GENERAL JOURNAL',
                             'Date': date_list})

df_salestax = pd.DataFrame({'TRNSID': trnsid,
                            'Amount': control_sheet_df['Sales Tax'] * -1,
                            'Memo': 'Sales Tax',
                            'Class' : class_list,
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': date_list,
                            'Name': 'SBOE'})

df_footlong = pd.DataFrame({'TRNSID': trnsid,
                            'Amount': control_sheet_df['Footlong'] * -1,
                            'Memo': 'Footlong',
                            'Class' : class_list,
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': date_list})

df_sixinch = pd.DataFrame({'TRNSID': trnsid,
                           'Amount': control_sheet_df['Six Inch'] * -1, 
                           'Memo': 'Six Inch',
                           'Class' : class_list,
                           'TRNSTYPE': 'GENERAL JOURNAL',
                           'Date': date_list})

df_threeinch = pd.DataFrame({'TRNSID': trnsid,
                             'Amount': control_sheet_df['Three Inch'] *-1,
                             'Memo': 'Three Inch',
                             'Class' : class_list,
                             'TRNSTYPE': 'GENERAL JOURNAL',
                             'Date': date_list})

df_muffin = pd.DataFrame({'TRNSID': trnsid,
                          'Amount': control_sheet_df['Muffin'] *-1,
                          'Memo': 'Muffin',
                          'Class' : class_list,
                          'TRNSTYPE': 'GENERAL JOURNAL',
                          'Date': date_list})

df_salad = pd.DataFrame({'TRNSID': trnsid,
                         'Amount': control_sheet_df['Salad'] *-1, 
                         'Memo': 'Salad',
                         'Class' : class_list,
                         'TRNSTYPE': 'GENERAL JOURNAL',
                         'Date': date_list})

df_pizza = pd.DataFrame({'TRNSID': trnsid,
                         'Amount': control_sheet_df['Pizza'] *-1, 
                         'Memo': 'Pizza',
                         'Class' : class_list,
                         'TRNSTYPE': 'GENERAL JOURNAL',
                         'Date': date_list})

df_othercarrier = pd.DataFrame({'TRNSID': trnsid,
                                'Amount': control_sheet_df['Other Carrier'] *-1,
                                'Memo': 'Other Carier',
                                'Class' : class_list,
                                'TRNSTYPE': 'GENERAL JOURNAL',
                                'Date': date_list})

df_addon = pd.DataFrame({'TRNSID': trnsid,
                         'Amount': control_sheet_df['Add On'] *-1,
                         'Memo': 'Add On',
                         'Class' : class_list,
                         'TRNSTYPE': 'GENERAL JOURNAL',
                         'Date': date_list})

df_catering_1 = pd.DataFrame({'TRNSID': trnsid,
                              'Amount': control_sheet_df['Catering'] *-1,
                              'Memo': 'Catering',
                              'Class' : class_list,
                              'TRNSTYPE': 'GENERAL JOURNAL',
                              'Date': date_list})

df_unitcouponsdisc = pd.DataFrame({'TRNSID': trnsid,
                                   'Amount': control_sheet_df['Unit Coupons Disc.'] *-1,
                                   'Memo': 'Unit Coupon Disc',
                                   'Class' : class_list,
                                   'TRNSTYPE': 'GENERAL JOURNAL',
                                   'Date': date_list})

df_unitrefunds = pd.DataFrame({'TRNSID': trnsid,
                               'Amount': control_sheet_df['Unit Refunds'] *-1,
                               'Memo': 'Unit Refunds',
                               'Class' : class_list,
                               'TRNSTYPE': 'GENERAL JOURNAL',
                               'Date': date_list})

df_unitvoids = pd.DataFrame({'TRNSID': trnsid,
                             'Amount': control_sheet_df['Unit Voids'] *-1,
                             'Memo': 'Unit Voids',
                             'Class' : class_list,
                             'TRNSTYPE': 'GENERAL JOURNAL',
                             'Date': date_list})

df_fountain = pd.DataFrame({'TRNSID': trnsid,
                            'Amount': control_sheet_df['Fountain'] *-1,
                            'Memo': 'Fountain',
                            'Class' : class_list,
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': date_list})

df_bottledbeverage = pd.DataFrame({'TRNSID': trnsid,
                                   'Amount': control_sheet_df['Bottled Beverage'] *-1,
                                   'Memo': 'Bottle Beverage',
                                   'Class' : class_list,
                                   'TRNSTYPE': 'GENERAL JOURNAL',
                                   'Date': date_list})

df_hotbeverage = pd.DataFrame({'TRNSID': trnsid,
                               'Amount': control_sheet_df['Hot Beverage'] *-1,
                               'Memo': 'Hot Beverage',
                               'Class' : class_list,
                               'TRNSTYPE': 'GENERAL JOURNAL',
                               'Date': date_list})

df_otherbeverage = pd.DataFrame({'TRNSID': trnsid,
                                 'Amount': control_sheet_df['Other Beverage'] *-1,
                                 'Memo': 'Other Beverage',
                                 'Class' : class_list,
                                 'TRNSTYPE': 'GENERAL JOURNAL',
                                 'Date': date_list})

df_drinkscouponsdisc = pd.DataFrame({'TRNSID': trnsid,
                                     'Amount': control_sheet_df['Drinks Coupons Disc.'] *-1,
                                     'Memo': 'Drinks Coupons Disc.',
                                     'Class' : class_list,
                                     'TRNSTYPE': 'GENERAL JOURNAL',
                                     'Date': date_list})

df_drinksrefund = pd.DataFrame({'TRNSID': trnsid,
                                'Amount': control_sheet_df['Drinks Refunds'] *-1,
                                'Memo': 'Drinks Refunds',
                                'Class' : class_list,
                                'TRNSTYPE': 'GENERAL JOURNAL',
                                'Date': date_list})

df_drinksvoids = pd.DataFrame({'TRNSID': trnsid,
                               'Amount': control_sheet_df['Drinks Voids'] *-1,
                               'Memo': 'Drinks Voids',
                               'Class' : class_list,
                               'TRNSTYPE': 'GENERAL JOURNAL',
                               'Date': date_list})

df_chips = pd.DataFrame({'TRNSID': trnsid,
                         'Amount': control_sheet_df['Chips'] *-1,
                         'Memo': 'Chips',
                         'Class' : class_list,
                         'TRNSTYPE': 'GENERAL JOURNAL',
                         'Date': date_list})

df_cookies = pd.DataFrame({'TRNSID': trnsid,
                           'Amount': control_sheet_df['Cookies'] *-1,
                           'Memo': 'Cookies',
                           'Class' : class_list,
                           'TRNSTYPE': 'GENERAL JOURNAL',
                           'Date': date_list})

df_soups = pd.DataFrame({'TRNSID': trnsid,
                         'Amount': control_sheet_df['Soups'] *-1,
                         'Memo': 'Soups',
                         'Class' : class_list,
                         'TRNSTYPE': 'GENERAL JOURNAL',
                         'Date': date_list})

df_othermisc = pd.DataFrame({'TRNSID': trnsid,
                             'Amount': control_sheet_df['Other Misc'] *-1,
                             'Memo': 'Other Misc',
                             'Class' : class_list,
                             'TRNSTYPE': 'GENERAL JOURNAL',
                             'Date': date_list})

df_othercouponsdisc = pd.DataFrame({'TRNSID': trnsid,
                                    'Amount': control_sheet_df['Other Coupons Disc.'] *-1,
                                    'Memo': 'Other Coupons Disc.',
                                    'Class' : class_list,
                                    'TRNSTYPE': 'GENERAL JOURNAL',
                                    'Date': date_list})

df_otherrefunds = pd.DataFrame({'TRNSID': trnsid,
                                'Amount': control_sheet_df['Other Refunds'] *-1,
                                'Memo': 'Other Refunds',
                                'Class' : class_list,
                                'TRNSTYPE': 'GENERAL JOURNAL',
                                'Date': date_list})

df_othervoids = pd.DataFrame({'TRNSID': trnsid,
                              'Amount': control_sheet_df['Other Voids'] *-1,
                              'Memo': 'Other Voids',
                              'Class' : class_list,
                              'TRNSTYPE': 'GENERAL JOURNAL',
                              'Date': date_list})

df_otherreceipts = pd.DataFrame({'TRNSID': trnsid,
                                 'Amount': control_sheet_df['Other Receipts'] *-1,
                                 'Memo': 'Other Receipts',
                                 'Class' : class_list,
                                 'TRNSTYPE': 'GENERAL JOURNAL',
                                 'Date': date_list})


df_thirdpartydelivery = pd.DataFrame({'TRNSID': delivery_trnsid,
                            'Amount': delivery_data_df['Daily Delivery Sum'],
                            'Memo': 'Third Party Delivery', 
                            'Class' : delivery_trnsid, 
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': delivery_trnsid})

df_doordash = pd.DataFrame({'TRNSID': delivery_trnsid,
                            'Amount': delivery_data_df['Tender DEL-DOORDASH'],
                            'Memo': 'Doordash', 
                            'Class' : delivery_trnsid, 
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': delivery_trnsid})

df_grubhub = pd.DataFrame({'TRNSID': delivery_trnsid,
                            'Amount': delivery_data_df['Tender DEL-Grubhub'],
                            'Memo': 'GrubHub', 
                            'Class' : delivery_trnsid, 
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': delivery_trnsid})

df_postmates = pd.DataFrame({'TRNSID': delivery_trnsid,
                            'Amount': delivery_data_df['Tender DEL-Postmates'],
                            'Memo': 'Postmates', 
                            'Class' : delivery_trnsid, 
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': delivery_trnsid})

df_ubereats = pd.DataFrame({'TRNSID': delivery_trnsid,
                            'Amount': delivery_data_df['Tender DEL-UberEatS'],
                            'Memo': 'Uber Eats', 
                            'Class' : delivery_trnsid, 
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': delivery_trnsid})

df_delivery_daily_sales = pd.DataFrame({'TRNSID': delivery_trnsid,
                                        'Amount': delivery_data_df['Tender DEL-DOORDASH'] + delivery_data_df['Tender DEL-Grubhub'] + delivery_data_df['Tender DEL-Postmates'] + delivery_data_df['Tender DEL-UberEatS'],
                                        'Memo': 'Daily Delivery Sales Offset',
                                        'Class' : delivery_trnsid, 
                                        'TRNSTYPE': 'GENERAL JOURNAL',
                                        'Date': delivery_trnsid})

df_doordash_sales = pd.DataFrame({'TRNSID': delivery_trnsid,
                            'Amount': delivery_data_df['Tender DEL-DOORDASH'] * -1,
                            'Memo': 'Doordash-Sales', 
                            'Class' : delivery_trnsid, 
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': delivery_trnsid})

df_grubhub_sales = pd.DataFrame({'TRNSID': delivery_trnsid,
                            'Amount': delivery_data_df['Tender DEL-Grubhub']  * -1,
                            'Memo': 'GrubHub-Sales', 
                            'Class' : delivery_trnsid, 
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': delivery_trnsid})

df_postmates_sales = pd.DataFrame({'TRNSID': delivery_trnsid,
                            'Amount': delivery_data_df['Tender DEL-Postmates'] * -1,
                            'Memo': 'Postmates-Sales', 
                            'Class' : delivery_trnsid, 
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': delivery_trnsid})

df_ubereats_sales = pd.DataFrame({'TRNSID': delivery_trnsid,
                            'Amount': delivery_data_df['Tender DEL-UberEatS'] *-1,
                            'Memo': 'Uber Eats-Sales', 
                            'Class' : delivery_trnsid, 
                            'TRNSTYPE': 'GENERAL JOURNAL',
                            'Date': delivery_trnsid})

control_sheet_result = df_deposits.append([df_batchimport,
                                           df_gift_card_catering,
                                           df_vmd_credit_sales, 
                                           df_amex_credit_sales,
                                           df_paid_outs,
                                           df_paypal_1, 
                                           df_over_short, 
                                           df_opencash, 
                                           df_closekeep,
                                           df_salestax, 
                                           df_footlong, 
                                           df_sixinch, 
                                           df_threeinch, 
                                           df_muffin,
                                           df_salad,
                                           df_pizza,
                                           df_othercarrier,
                                           df_addon,
                                           df_catering_1,
                                           df_unitcouponsdisc,
                                           df_unitrefunds,
                                           df_unitvoids,
                                           df_fountain,
                                           df_bottledbeverage,
                                           df_hotbeverage,
                                           df_otherbeverage,
                                           df_drinkscouponsdisc,
                                           df_drinksrefund,
                                           df_drinksvoids,
                                           df_chips,
                                           df_cookies,
                                           df_soups,
                                           df_othermisc,
                                           df_othercouponsdisc,
                                           df_otherrefunds,
                                           df_othervoids,
                                           df_otherreceipts,
                                           df_doordash,
                                           df_ubereats,
                                           df_postmates,
                                           df_grubhub,
                                           df_delivery_daily_sales,
                                           df_doordash_sales,
                                           df_grubhub_sales,
                                           df_postmates_sales,
                                           df_ubereats_sales])
                    

control_sheet_result = control_sheet_result[['!TRNS','TRNSID', 'TRNSTYPE', 'Date', 'Amount', 'Memo', 'Class', 'Name']]

control_sheet_result = control_sheet_result.sort_values(by = ['Class', 'Date', '!TRNS'])

control_sheet_result

#writer = pd.ExcelWriter('/Users/rginsberg/Downloads/Subway-Paul Chopra/Results/Control_Sheet_Results-V1.xlsx')
#control_sheet_result.to_excel(writer,'Sheet1', index=False)
#writer.save()

control_sheet_result

,!TRNS,TRNSID,TRNSTYPE,Date,Amount,Memo,Class,Name
0,NaN,2019-03-01 00:00:00,GENERAL JOURNAL,2019-03-01 00:00:00,27.55,Doordash,2019-03-01 00:00:00,NaN
31,NaN,2019-03-01 00:00:00,GENERAL JOURNAL,2019-03-01 00:00:00,0,Doordash,2019-03-01 00:00:00,NaN
62,NaN,2019-03-01 00:00:00,GENERAL JOURNAL,2019-03-01 00:00:00,0,Doordash,2019-03-01 00:00:00,NaN
93,NaN,2019-03-01 00:00:00,GENERAL JOURNAL,2019-03-01 00:00:00,0,Doordash,2019-03-01 00:00:00,NaN
124,NaN,2019-03-01 00:00:00,GENERAL JOURNAL,2019-03-01 00:00:00,0,Doordash,2019-03-01 00:00:00,NaN
155,NaN,2019-03-01 00:00:00,GENERAL JOURNAL,2019-03-01 00:00:00,15.25,Doordash,2019-03-01 00:00:00,NaN
186,NaN,2019-03-01 00:00:00,GENERAL JOURNAL,2019-03-01 00:00:00,0,Doordash,2019-03-01 00:00:00,NaN
217,NaN,2019-03-01 00:00:00,GENERAL JOURNAL,2019-03-01 00:00:00,0,Doordash,2019-03-01 00:00:00,NaN
248,NaN,2019-03-01 00:00:00,GENERAL JOURNAL,2019-03-01 00:00:00,56.9,Doordash,2019-03-01 00:00:00,NaN
279,NaN,2019-03-01 00:00:00,GENERAL JOURNAL,2019-03-01 00:00:00,33.41,Doordash,2019-03-01 00:00:00,NaN
